In [1]:
pip install beautifulsoup4 requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 618.1 kB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [31]:
import requests
from bs4 import BeautifulSoup
import csv
from datetime import datetime
import os
import pandas as pd

In [3]:


def crawl_ptt_nissan(search_keyword='Nissan'):
    url = 'https://www.ptt.cc/bbs/car/search?q=' + search_keyword

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        articles = soup.find_all('div', class_='r-ent')

        data_list = []

        for article in articles:
            title = article.select_one('.title a').text.strip()
            link = 'https://www.ptt.cc' + article.select_one('.title a')['href']
            article_data = crawl_article(link)

            if article_data:
                data_list.append({
                    'title': title,
                    'time': article_data['post_time'],
                    'content': article_data['post_content'],
                    'comments': article_data['comments']
                })

        return data_list
    else:
        print('Failed to retrieve the search page. Status code:', response.status_code)
        return None

def crawl_article(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        post_title = soup.select_one('.article-meta-value:nth-of-type(3)').text
        post_time = soup.select_one('.article-meta-value:nth-of-type(4)').text
        post_content = soup.select_one('#main-content').text

        comments = []
        comment_elements = soup.select('.push')
        for comment_element in comment_elements:
            push_tag = comment_element.select_one('.push-tag').text.strip()
            push_userid = comment_element.select_one('.push-userid').text.strip()
            push_content = comment_element.select_one('.push-content').text.strip()
            comments.append({'push_tag': push_tag, 'push_userid': push_userid, 'push_content': push_content})

        return {
            'post_time': post_time,
            'post_content': post_content,
            'comments': comments
        }
    else:
        print('Failed to retrieve the article page. Status code:', response.status_code)
        return None

def save_to_csv(data, folder_path='output'):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    timestamp = datetime.now().strftime('%Y%m%d%H%M%S')
    filename = f'{folder_path}/ptt_nissan_{timestamp}.csv'

    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['title', 'time', 'content', 'comments']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        writer.writerows(data)

if __name__ == "__main__":
    crawled_data = crawl_ptt_nissan()

    if crawled_data:
        save_to_csv(crawled_data)
        print(f'Data saved to the output folder.')
    else:
        print('Data crawling failed.')

AttributeError: 'NoneType' object has no attribute 'text'

In [37]:

# 設定 Ptt Nissan 板的 URL
url = 'https://www.ptt.cc/bbs/Nissan/index.html'

# 設定 headers，模擬瀏覽器請求
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}

# 建立一個資料夾存放 CSV 檔案
output_folder = '/Users/isc_admin/Desktop/網頁爬蟲'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 建立 CSV 檔案的路徑
csv_file_path = os.path.join(output_folder, 'nissan_articles.csv')

# 開啟 CSV 檔案，建立 CSV 寫入器
with open(csv_file_path, 'w', newline='', encoding='utf-8') as csvfile:
    # 設定 CSV 檔案的欄位名稱
    fieldnames = ['文章標題', '發文時間', '文章內容', '留言']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    # 寫入 CSV 檔案的標頭
    writer.writeheader()

    # 爬取 Ptt Nissan 板的頁面
    for page in range(5):  # 只爬取前 5 頁，你可以根據需求調整
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')

        # 找到文章列表中的每一篇文章
        articles = soup.find_all('div', class_='r-ent')
        for article in articles:
            try:
                title_element = article.find('div', class_='title')
                title = title_element.text.strip() if title_element else 'N/A'

                date_element = article.find('div', class_='date')
                date = date_element.text.strip() if date_element else 'N/A'

                link_element = article.find('a')
                link = 'https://www.ptt.cc' + link_element['href'] if link_element else 'N/A'

                # 進入文章頁面爬取內容
                article_response = requests.get(link, headers=headers)
                article_soup = BeautifulSoup(article_response.text, 'html.parser')
                content = article_soup.find('div', id='main-content').text if article_soup.find('div', id='main-content') else 'N/A'
                comments = article_soup.find_all('div', class_='push') if article_soup.find_all('div', class_='push') else []

                # 寫入 CSV 檔案
                writer.writerow({
                    '文章標題': title,
                    '發文時間': date,
                    '文章內容': content,
                    '留言': comments
                })
                
            except Exception as e:
                print(f"Error: {e}")
        

print(f"爬取完成，結果已儲存至 {csv_file_path}")

爬取完成，結果已儲存至 /Users/isc_admin/Desktop/網頁爬蟲/nissan_articles.csv


In [41]:
article_response = requests.get(link, headers=headers)
article_soup = BeautifulSoup(article_response.text, 'html.parser')
content = article_soup.find('div', id='main-content').text if article_soup.find('div', id='main-content') else 'N/A'
comments = article_soup.find_all('div', class_='push') if article_soup.find_all('div', class_='push') else []

comments

InvalidSchema: No connection adapters were found for "['https://www.ptt.cc/bbs/car/M.1699346704.A.19F.html', 'https://www.ptt.cc/bbs/car/M.1698981078.A.EBE.html', 'https://www.ptt.cc/bbs/car/M.1698466656.A.BAD.html', 'https://www.ptt.cc/bbs/car/M.1698388504.A.40F.html', 'https://www.ptt.cc/bbs/car/M.1698386424.A.210.html', 'https://www.ptt.cc/bbs/car/M.1698385231.A.FCD.html', 'https://www.ptt.cc/bbs/car/M.1698366402.A.8FE.html', 'https://www.ptt.cc/bbs/car/M.1698090594.A.0DE.html', 'https://www.ptt.cc/bbs/car/M.1698031323.A.C2C.html', 'https://www.ptt.cc/bbs/car/M.1698026322.A.5D2.html', 'https://www.ptt.cc/bbs/car/M.1697995870.A.273.html', 'https://www.ptt.cc/bbs/car/M.1697945931.A.F16.html', 'https://www.ptt.cc/bbs/car/M.1697466809.A.309.html', 'https://www.ptt.cc/bbs/car/M.1697295225.A.130.html', 'https://www.ptt.cc/bbs/car/M.1696236980.A.406.html', 'https://www.ptt.cc/bbs/car/M.1696218471.A.30F.html', 'https://www.ptt.cc/bbs/car/M.1695452260.A.AB6.html', 'https://www.ptt.cc/bbs/car

In [73]:
#網路爬蟲-HTML格式讀取 
dataset = pd.DataFrame()
for page in range(1,2): 
    import urllib.request as request 
    search_keyword = 'Nissan'
    src='https://www.ptt.cc/bbs/car/search?page=' + str(page) + '&q='+ search_keyword #ptt salary版
    requestUA=request.Request(src, headers={
        "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.182 Safari/537.36"
    })
    with request.urlopen(requestUA) as response:
        data=response.read().decode("utf-8")
    import bs4
    root=bs4.BeautifulSoup(data, "html.parser") #讓BeautifulSoup協助解析HTML格式文件
    #印出網頁標題
    print(root.title.string)#尋找"title"標籤字串

    #印出文章主題
    titles=root.find_all("div", class_="title") #尋找所有class="title"的div標籤
    for title in titles:
        if title.a != None: #如果標題包含a標籤 (沒有被刪除), 印出來
            print(title.a.string)
    #抓取看板文章的連結
    #找到所有屬性class = "r-ent"
    rent = root.find_all('div',class_='r-ent')
    # print(ment)
    #建立一個空的陣列，把抓到的文章連結一個一個添加進去
    link=[]
    for title in rent:
        #由於有些被刪除的文章會抓不到連結，所以把抓不到被刪除的文章濾掉
        if title.a != None:
            link.append("https://www.ptt.cc"+title.a.get("href"))
    print(link)
    #取得文章連結
    #抓取文章內容
    for websites in link:
    #     src=websites #ptt salary版
        requestUA=request.Request(websites, headers={
            "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.182 Safari/537.36"
        })
        with request.urlopen(requestUA) as response:
            data=response.read().decode("utf-8")
        import bs4
        root=bs4.BeautifulSoup(data, "html.parser") #讓BeautifulSoup協助解析HTML格式文件
    #印出網頁標題
        print("===========================================================================================")
        print(root.title.string+'\n')#尋找"title"標籤字串
        header = root.find_all('span','article-meta-value')
        if header != []:
    #         print(header)
    # 作者
            author = header[0].text
            print("Author : "+ author+'\n')
    # 日期
            date = header[3].text
            print("Date : "+ date+'\n')
        print("==========================================================================================="+'\n')
    #印出文章主題
        titles=root.find_all("div", id_="main-content") 
        for title in titles:
            if title.a != None: #如果標題包含a標籤 (沒有被刪除), 印出來
                print(title.a.string)

        main_container = root.find(id='main-container')
    # 把所有文字都抓出來
        if main_container != " ":
            all_text = main_container.text
    # 把整個內容切割透過 "-- " 切割成2個陣列
            pre_text = all_text.split('--')[0]

    # 把每段文字 根據 '\n' 切開
            texts = pre_text.split('\n')
    # 如果你爬多篇你會發現 
            contents = texts[2:]
    # 內容
            content = '\n'.join(contents)

        print(content)

        pushs=root.find_all(class_="f3 push-content") 
    # print(pushs)
        push_userid=root.find_all(class_="f3 hl push-userid")
    # print(push_userid)

        id=[]
        for userid in push_userid:
            if userid != None: 
                id.append(userid.string)
    #         print(userid.string) 

        comment=[]
        for push in pushs:
            if push != None: 
                comment.append(push.string)
    #         print(push.string) 

        print("---------------push comment---------------"+"\n")
        id_comment=[]
        for i in range(len(id)):
            if id[i] and comment[i] != None:
                id_comment.append(id[i]+comment[i])
        #print(id_comment)
        cell_value = '\n'.join(id_comment)

        print('\n'+"-------------------------------End of article---------------------------------------------"+'\n'+'\n') 
        dataset = dataset.append(pd.DataFrame(data={'i': page,
                                                    'date':date,
                                                    'title':title.a.string,
                                                    'link':websites,
                                                    'author':author,
                                                    'content':content,
                                                    'comment':cell_value
                                                    }, index = [0]), ignore_index = True)

Nissan - car - 批踢踢實業坊
[電車] Nissan Ariya 
[新聞]不是特斯拉,日本最熱賣電動車Nissan Sakura「人人都買得起」!
Re: [討論] Nissan的車子484值得買啊
[討論] Nissan的車子484值得買啊
[討論] Nissan caravan 露營車發表
[分享] Nissan 電動概念車
[新聞] 改款「新Nissan Sentra」台灣現身！有望
Re: [新聞] 新車發表NISSAN KICKS e-POWER
Re: [心得] Nissan Kicks e-power 短程試駕體驗分享
[問題] Nissan的這是什麼按鍵？
[心得] Nissan Kicks e-power 短程試駕體驗分享
Re: [新聞] 新車發表NISSAN KICKS e-POWER
[菜單] NISSAN KICKS 旗艦版
[新聞] Nissan 小改款 X-Trail 意外提前曝光！
Re: [新聞] 104.9萬 全新NISSAN KICKS e-POWER
[新聞] 新車發表NISSAN KICKS e-POWER
[菜單] Nissan X-trail輕油電旗艦榮耀版
[新聞] 比亞迪在日本賣不好？他們更愛這款50萬級距的Nissan Sakura
[新聞] Nissan X-Trail 1.5 VC-Turbo正式發表
[菜單] Nissan Kicks 旗艦特仕版2023年式
['https://www.ptt.cc/bbs/car/M.1699346704.A.19F.html', 'https://www.ptt.cc/bbs/car/M.1698981078.A.EBE.html', 'https://www.ptt.cc/bbs/car/M.1698466656.A.BAD.html', 'https://www.ptt.cc/bbs/car/M.1698388504.A.40F.html', 'https://www.ptt.cc/bbs/car/M.1698386424.A.210.html', 'https://www.ptt.cc/bbs/car/M.1698385231.A.FCD.html', 'https://www.ptt.cc/bbs/car/M.1698366402.A.8FE.ht

Re: [新聞] 新車發表NISSAN KICKS e-POWER - 看板 car - 批踢踢實業坊

Author : JG8861 ((冷氣終於來裝了))

Date : Tue Oct 24 03:49:52 2023


※ 引述《JG8861 ((冷氣終於來裝了))》之銘言：
: 若是出小一點的車型(cc)，e-POWER的車，
: 價格是否能壓低到70～80萬呢？


日本國內有啊！

https://www.kingautos.net/305855
https://www.carnews.com/feature_story/newcar/336300
https://www.carstuff.com.tw/car-news/item/32980-e-power-nissan-note.html


---------------push comment---------------


-------------------------------End of article---------------------------------------------


Re: [心得] Nissan Kicks e-power 短程試駕體驗分享 - 看板 car - 批踢踢實業坊

Author : kingshapo (Shapo!)

Date : Mon Oct 23 11:22:01 2023


e power: 不用充電的電動車 怎麼看怎麼怪
意思是用95無鉛去幫電池充電 然後再用電去開車
所以整部車還是汽油帶你走 只是中間加了電池 假裝自己是電車

是不是跟戴保險套看A片一樣 假裝自己有在做愛

這nissan獨家科技 過不久就會被淘汰了吧 就像之前很煩躁的怠速停火

請問: 目前x-trail e-power 接受度高嗎? nissan這麼有信心 又出了kicks e power應
該是賣得很好吧

※ 引述《louie83279 (永遠不準的先知)》之銘言：
: 這篇算是預期之外的一次試駕體驗(原本要先寫Opel Mokka)，所以可能比較簡短，就寫一
: 點關於開過Kicks e-power之後的感想就好，不多作介紹了，有興趣看內裝配備詳解的朋
: 友，請看媒體拍比較仔細。
: https://i.imgur.com/PL0dbJ

[新聞] 104.9萬 全新NISSAN KICKS e-POWER - 看板 car - 批踢踢實業坊

Author : mantcjk (小柏)

Date : Mon Oct  2 11:47:49 2023




原文連結：
https://www.7car.tw/articles/read/96644

原文內容：
裕隆日產今日 (10/02) 宣布 Nissan Kicks e-POWER 即將登場，採單一車規編成，建議預售價 104.9 萬元，自即日起至 2023 年 10 月 17 日止啟動線上預售。

參照海外市場販售的 Nissan Kicks e-POWER，動力最大輸出為 129hp、260Nm，採用壓縮比達 12:1 以上的 HR12DE 1.2L 三缸直噴自然進氣汽油引作為發電機，為鋰電池充電，搭配可驅動/回收電能發電的「EM57」電動馬達，解決純電車的費時充電問題之餘，同時兼備跟純電車款一樣的電動駕駛感及比一般 Hybrid 車還要節能油耗表現，搭配 Nissan e-Pedal 單踏板控制技術，使駕駛者僅用油門踏板就能達成加速及減速。


心得/說明：(30字以上)
104.9萬價格好像有點競爭力，應該可以搶到油電CC的客群，但打的到Crv X-trail Tyson L嗎？

---------------push comment---------------


-------------------------------End of article---------------------------------------------


[菜單] Nissan X-trail輕油電旗艦榮耀版 - 看板 car - 批踢踢實業坊

Author : taco85216 (跟你媽媽講哦)

Date : Sat Sep 23 14:57:38 2023




車輛廠牌/年份/型號：Nissan Xtrail輕油電旗艦榮耀版2023年式

車輛原廠編成等級：旗艦版中的榮耀版

是否含保險：否

是否含領牌稅金：否

成交價格(總價):128萬（雙色車身1.5w+前擋升級冰鑽s75 0.5w+行車記錄器升級Mio 0.1w）

空車價格：125.9萬（牌價130.9，折五萬）

下訂日期範圍：8/4下訂，預計10/6交車



In [74]:
dataset
dataset.to_csv('/Users/isc_admin/Desktop/網頁爬蟲/nissan_articles.csv', index=False, encoding='utf-8')


In [66]:
content

'去美國的時候朋友剛好想要換車\n\n用lease的方式 先繳4000後每月499簽3年XD\n\n後座空間感OK  行李箱小 一線燈吸睛\n\n加速也是很有貼背感\n\n初看還以為是porsche\nhttps://imgur.com/jpBe5Y5\nhttps://imgur.com/FtZj4rJ\nhttps://imgur.com/27ElNWk\n\n'

In [42]:
#網路爬蟲-HTML格式讀取 
dataset = pd.DataFrame()
search_keyword = 'Nissan'
src='https://www.ptt.cc/bbs/car/search?page=' + str(page) + '&q='+ search_keyword #ptt salary版
requestUA=request.Request(src, headers={
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.182 Safari/537.36"
})
with request.urlopen(requestUA) as response:
    data=response.read().decode("utf-8")
    


In [69]:
pushs=root.find_all(class_="f3 push-content") 
# print(pushs)
push_userid=root.find_all(class_="f3 hl push-userid")
# print(push_userid)

id=[]
for userid in push_userid:
    if userid != None: 
        id.append(userid.string)
#print(userid.string) 

comment=[]
for push in pushs:
    if push != None: 
        comment.append(push.string)
    #print(push.string) 

print("---------------push comment---------------"+"\n")
id_comment=[]
for i in range(len(id)):
    if id[i] and comment[i] != None:
        id_comment.append(id[i]+comment[i])
print(id_comment)
cell_value = '\n'.join(id_comment)   
cell_value


---------------push comment---------------

['keikochen   : 他', 'sjerrysss   : 遲早被Mazda取代', 'louisxxiii  : Mazda? 要確定欸', 'a0921387223 : 這台好像蠻香的 馬自達？ 坐垃圾電車那個品牌嗎', 'blitz1991   : 破鞋？ 眼睛用粗蠟推過？', 'giveme520   : 總比嘴巴生瘡的好', 'ppokok      : 馬自達！！您是說純電MX-30做出來只賣了不到一百台', 'ppokok      : 那個日廠嗎？', 'ray5607     : 好看有一點豪華感', 'sbsame12455 : 那個雨刷真的好突兀= =', 'joiedevivre : 心目中最美電動客車 唯一可惜就是玉龍', 'andy0121    : 內裝很有質感！', 'andy0121    : 進來台灣不知道又會被拔什麼', 'h2hero1975  : 這台很不錯', 'JokerAwesome: 這台我記得就是松隆子和木村拓哉一起代言那台？', 'hmt17       : 帥', 'woodyzhou   : 這台間諜測試車今年5月有在台灣路上捕獲過，沒意外', 'woodyzhou   : 應該明年會開賣', 'hanchueh    : 一年賣不到5000台的稀有車', 'kuff220     : 這台真的帥到哭', 'qf852741963 : 有點像harrier', 'jei01       : Nissan 電車算是日廠做得比較成功的一家', 'ShaNe1993   : 背後根本NX', 'ShaNe1993   : 可惜價格太高，被Y壓在地上摩擦', 'ilovedirk41 : 這台美國售價是多少?', 'INTHEWIND   : 你的鏡頭很油', 'ShaNe1993   : 43190usd-60190usd', 'ShaNe1993   : 對標同級Y的規格 幾乎都貴一點', 'ilovedirk41 : 美國就比y貴？那台灣不用想了', 'garlicbap   : 這台會進啊不過是CUV後座不行', 'p5auser     : 每月499鎂，太便宜了，台灣怎麼比', 'lee1023     : 

'keikochen   : 他\nsjerrysss   : 遲早被Mazda取代\nlouisxxiii  : Mazda? 要確定欸\na0921387223 : 這台好像蠻香的 馬自達？ 坐垃圾電車那個品牌嗎\nblitz1991   : 破鞋？ 眼睛用粗蠟推過？\ngiveme520   : 總比嘴巴生瘡的好\nppokok      : 馬自達！！您是說純電MX-30做出來只賣了不到一百台\nppokok      : 那個日廠嗎？\nray5607     : 好看有一點豪華感\nsbsame12455 : 那個雨刷真的好突兀= =\njoiedevivre : 心目中最美電動客車 唯一可惜就是玉龍\nandy0121    : 內裝很有質感！\nandy0121    : 進來台灣不知道又會被拔什麼\nh2hero1975  : 這台很不錯\nJokerAwesome: 這台我記得就是松隆子和木村拓哉一起代言那台？\nhmt17       : 帥\nwoodyzhou   : 這台間諜測試車今年5月有在台灣路上捕獲過，沒意外\nwoodyzhou   : 應該明年會開賣\nhanchueh    : 一年賣不到5000台的稀有車\nkuff220     : 這台真的帥到哭\nqf852741963 : 有點像harrier\njei01       : Nissan 電車算是日廠做得比較成功的一家\nShaNe1993   : 背後根本NX\nShaNe1993   : 可惜價格太高，被Y壓在地上摩擦\nilovedirk41 : 這台美國售價是多少?\nINTHEWIND   : 你的鏡頭很油\nShaNe1993   : 43190usd-60190usd\nShaNe1993   : 對標同級Y的規格 幾乎都貴一點\nilovedirk41 : 美國就比y貴？那台灣不用想了\ngarlicbap   : 這台會進啊不過是CUV後座不行\np5auser     : 每月499鎂，太便宜了，台灣怎麼比\nlee1023     : 明年底ㄅ但是看價錢決定\nwoodyzhou   : Ariya車長約4600mm，跟RAV4差不多長，軸距2775mm，\nwoodyzhou   : 軸距比Tucson L還要長\nkrej5566    : 

In [58]:
id_comment

'edweepvp    : 沒有盤不盤 自己喜歡就好'

In [ ]:
#印出文章主題
titles=root.find_all("div", class_="title") #尋找所有class="title"的div標籤
for title in titles:
    if title.a != None: #如果標題包含a標籤 (沒有被刪除), 印出來
        print(title.a.string)
    #抓取看板文章的連結
    #找到所有屬性class = "r-ent"
rent = root.find_all('div',class_='r-ent')
    # print(ment)
    #建立一個空的陣列，把抓到的文章連結一個一個添加進去
    link=[]
    for title in rent:
        #由於有些被刪除的文章會抓不到連結，所以把抓不到被刪除的文章濾掉
        if title.a != None:
            link.append("https://www.ptt.cc"+title.a.get("href"))
    print(link)
    #取得文章連結
    #抓取文章內容
    for websites in link:
    #     src=websites #ptt salary版
        requestUA=request.Request(websites, headers={
            "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.182 Safari/537.36"
        })
        with request.urlopen(requestUA) as response:
            data=response.read().decode("utf-8")
        import bs4
        root=bs4.BeautifulSoup(data, "html.parser") #讓BeautifulSoup協助解析HTML格式文件
    #印出網頁標題
        print("===========================================================================================")
        print(root.title.string+'\n')#尋找"title"標籤字串
        header = root.find_all('span','article-meta-value')
        if header != []:
    #         print(header)
    # 作者
            author = header[0].text
            print("Author : "+ author+'\n')
    # 日期
            date = header[3].text
            print("Date : "+ date+'\n')
        print("==========================================================================================="+'\n')
    #印出文章主題
        titles=root.find_all("div", id_="main-content") 
        for title in titles:
            if title.a != None: #如果標題包含a標籤 (沒有被刪除), 印出來
                print(title.a.string)

        main_container = root.find(id='main-container')
    # 把所有文字都抓出來
        if main_container != " ":
            all_text = main_container.text
    # 把整個內容切割透過 "-- " 切割成2個陣列
            pre_text = all_text.split('--')[0]

    # 把每段文字 根據 '\n' 切開
            texts = pre_text.split('\n')
    # 如果你爬多篇你會發現 
            contents = texts[2:]
    # 內容
            content = '\n'.join(contents)

        print(content)

        pushs=root.find_all(class_="f3 push-content") 
    # print(pushs)
        push_userid=root.find_all(class_="f3 hl push-userid")
    # print(push_userid)

        id=[]
        for userid in push_userid:
            if userid != None: 
                id.append(userid.string)
    #         print(userid.string) 

        comment=[]
        for push in pushs:
            if push != None: 
                comment.append(push.string)
    #         print(push.string) 

        print("---------------push comment---------------"+"\n")
        for i in range(len(id)):
            if id[i] and comment[i] != None:
                print(id[i]+comment[i])

        print('\n'+"-------------------------------End of article---------------------------------------------"+'\n'+'\n') 
        dataset = dataset.append(pd.DataFrame(data={'i': page,
                                                    'date':date,
                                                    'title':title.a.string,
                                                    'author':author,
                                                    'content':content,
                                                    'comment':comment
                                                    }, index = [0]), ignore_index = True)

In [36]:
title.a.string

'[菜單] Nissan Kicks 旗艦特仕版2023年式'